In [1]:
import sympy as sp
from sympy import symbols
from sympy.physics import mechanics
from sympy import Derivative, Inverse

In [2]:
#-------------------------------------------------
# PENDULUM MODEL

# Number of segments
n = 3

# Generalized coordinates and velocities
# (angular positions & velocities of each mass) 
q = mechanics.dynamicsymbols('q:{0}'.format(n))
qd = mechanics.dynamicsymbols('q:{0}'.format(n), 1)
u = mechanics.dynamicsymbols('u:{0}'.format(n))
ud = mechanics.dynamicsymbols('u:{0}'.format(n), 1)

# mass and length of each 
m = symbols('m:{0}'.format(n))
l = symbols('l:{0}'.format(n))

# gravity and time symbols
g, t = symbols('g t')


#--------------------------------------------------
# INTEGRATION MODEL
# Create reference frame
# The reference frame will have the x axis pointing down, y axis pointing right and z axis pointing inwards the screen
K = mechanics.ReferenceFrame('K')

# Create point (which stores the position, velocity and acceleration)
P = mechanics.Point('P')

# Set to zero the velocity of the point in the reference frame K
P.set_vel(K, 0)

# Create lists to hold particles, forces and kinetic ODEs for each pendulum's segment
particles = []

# Run through all the segments:
for i in range(n):
    # Create a reference frame following the i^th mass
    # We call the new frames "Ki" (i=1...n)
    # The new frames follow the position of the i^th mass and are rotated around the z axis
    Ki = K.orientnew('K' + str(i), 'Axis', [q[i], K.z])

    # Set the angular velocity of the frame Ki with respect to the frame K 
    Ki.set_ang_vel(K, qd[i] * K.z)

    # Create a point in this reference frame
    Pi = P.locatenew('P' + str(i), l[i] * Ki.x)

    # Set the velocity of this point in K based on the velocity of P in Ki
    Pi.v2pt_theory(P, K, Ki)

    # Create a new particle of mass m[i] at this point
    Pai = mechanics.Particle('Pa' + str(i), Pi, m[i])

    # Potential energies
    for j in range(i+1):
        Pai.potential_energy += -1 * m[i] * g * ( l[j] * sp.cos(q[j]) )

    particles.append(Pai)

    
    P = Pi

In [3]:
# Generate equations of motion using Lagrange's Method
L = mechanics.Lagrangian(K, *particles)
LM = mechanics.LagrangesMethod(L, q)
eq = LM.form_lagranges_equations()

In [4]:
L

g*l0*m0*cos(q0(t)) + g*l0*m1*cos(q0(t)) + g*l0*m2*cos(q0(t)) + g*l1*m1*cos(q1(t)) + g*l1*m2*cos(q1(t)) + g*l2*m2*cos(q2(t)) + l0**2*m0*Derivative(q0(t), t)**2/2 + l0**2*m1*Derivative(q0(t), t)**2/2 + l0**2*m2*Derivative(q0(t), t)**2/2 + l0*l1*m1*(sin(q0(t))*sin(q1(t)) + cos(q0(t))*cos(q1(t)))*Derivative(q0(t), t)*Derivative(q1(t), t) + l0*l1*m2*(sin(q0(t))*sin(q1(t)) + cos(q0(t))*cos(q1(t)))*Derivative(q0(t), t)*Derivative(q1(t), t) + l0*l2*m2*(sin(q0(t))*sin(q2(t)) + cos(q0(t))*cos(q2(t)))*Derivative(q0(t), t)*Derivative(q2(t), t) + l1**2*m1*Derivative(q1(t), t)**2/2 + l1**2*m2*Derivative(q1(t), t)**2/2 + l1*l2*m2*(sin(q1(t))*sin(q2(t)) + cos(q1(t))*cos(q2(t)))*Derivative(q1(t), t)*Derivative(q2(t), t) + l2**2*m2*Derivative(q2(t), t)**2/2

In [5]:
eq.simplify()
eq

Matrix([
[l0*(g*m0*sin(q0(t)) + g*m1*sin(q0(t)) + g*m2*sin(q0(t)) + l0*m0*Derivative(q0(t), (t, 2)) + l0*m1*Derivative(q0(t), (t, 2)) + l0*m2*Derivative(q0(t), (t, 2)) + l1*m1*sin(q0(t) - q1(t))*Derivative(q1(t), t)**2 + l1*m1*cos(q0(t) - q1(t))*Derivative(q1(t), (t, 2)) + l1*m2*sin(q0(t) - q1(t))*Derivative(q1(t), t)**2 + l1*m2*cos(q0(t) - q1(t))*Derivative(q1(t), (t, 2)) + l2*m2*sin(q0(t) - q2(t))*Derivative(q2(t), t)**2 + l2*m2*cos(q0(t) - q2(t))*Derivative(q2(t), (t, 2)))],
[                                                    l1*(g*m1*sin(q1(t)) + g*m2*sin(q1(t)) - l0*m1*sin(q0(t) - q1(t))*Derivative(q0(t), t)**2 + l0*m1*cos(q0(t) - q1(t))*Derivative(q0(t), (t, 2)) - l0*m2*sin(q0(t) - q1(t))*Derivative(q0(t), t)**2 + l0*m2*cos(q0(t) - q1(t))*Derivative(q0(t), (t, 2)) + l1*m1*Derivative(q1(t), (t, 2)) + l1*m2*Derivative(q1(t), (t, 2)) + l2*m2*sin(q1(t) - q2(t))*Derivative(q2(t), t)**2 + l2*m2*cos(q1(t) - q2(t))*Derivative(q2(t), (t, 2)))],
[                                          

In [6]:
MM = LM.mass_matrix
FM = LM.forcing

In [13]:
qdd = (Inverse(MM)*FM).simplify()
qdd

Matrix([
[(m0/4 + m1/4 + m2/4)*(4*m2*(m1 + m2)*(((m1 + m2)*cos(q0(t) - q1(t))*cos(q0(t) - q2(t)) - (m0 + m1 + m2)*cos(q1(t) - q2(t)))*cos(q0(t) - q1(t)) + (m0 + m1 + m2 - (m1 + m2)*cos(q0(t) - q1(t))**2)*cos(q0(t) - q2(t)))*(-g*sin(q2(t)) + l0*sin(q0(t) - q2(t))*Derivative(q0(t), t)**2 + l1*sin(q1(t) - q2(t))*Derivative(q1(t), t)**2)*(m0 + m1 + m2 - (m1 + m2)*cos(q0(t) - q1(t))**2) - 4*(-m2*(((m1 + m2)*cos(q0(t) - q1(t))*cos(q0(t) - q2(t)) - (m0 + m1 + m2)*cos(q1(t) - q2(t)))*cos(q0(t) - q1(t)) + (m0 + m1 + m2 - (m1 + m2)*cos(q0(t) - q1(t))**2)*cos(q0(t) - q2(t)))*((m1 + m2)*cos(q0(t) - q1(t))*cos(q0(t) - q2(t)) - (m0 + m1 + m2)*cos(q1(t) - q2(t))) + (m2*((m1 + m2)*cos(q0(t) - q1(t))*cos(q0(t) - q2(t)) - (m0 + m1 + m2)*cos(q1(t) - q2(t)))**2 + (m1 + m2)*(-m0 - m1 + m2*cos(q0(t) - q2(t))**2 - m2)*(m0 + m1 + m2 - (m1 + m2)*cos(q0(t) - q1(t))**2))*cos(q0(t) - q1(t)))*(-g*m1*sin(q1(t)) - g*m2*sin(q1(t)) + l0*m1*sin(q0(t) - q1(t))*Derivative(q0(t), t)**2 + l0*m2*sin(q0(t) - q1(t))*Derivativ